In [1]:
from collections import defaultdict
from GrabReleaseCommits import *
from StarHistory import *
from GetRepoFromDataset import *
from CodeCovReport import *

In [ ]:
github_repos = filter_github_repos('../data/github-ranking-2024-02-15.csv')
refinedRepo = list()
for repo in github_repos:
    username, repo_name, token = repo[0], repo[1], "7848dd6f-5308-43f6-a02f-e10e31118854"
    repo_info = detect_coverage_tool_usage("github", username, repo_name, token)
    if repo_info != None:
        print(repo_info)
        refinedRepo.append(repo_info)

print(refinedRepo)
print(len(refinedRepo))

In [ ]:
Refined_DataSet = [['github', 'facebook', 'react', False, True], ['github', 'vuejs', 'vue', True, True], ['github', 'trekhleb', 'javascript-algorithms', True, None]]